In [42]:
import os
import warnings
import requests
import json

from openai import OpenAI
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage, UpstageLayoutAnalysisLoader
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_chroma import Chroma
from tavily import TavilyClient

warnings.filterwarnings("ignore")

%load_ext dotenv
%dotenv

client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
)
llm = ChatUpstage()
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
sentence = "헤어지는 마당에 뭘 이런 거까지 알리나 이를테면 그녀에게 흔들린 것 그건 정말이지 TMI"

In [18]:
sentence = "마지막 인사는 하지 마... 중꺾마."

In [19]:
sentence = "억까하지마, 이건 웃안웃이야!"

In [20]:
sentence = "더 이상 만나지 말라고 이건 킹리적 갓심이라고! 넌 갈비야!"

In [67]:
sentence = "우리 언니가 말없이 폰 붙들고 그딴 시답지 않은 개소리나 듣고 있길래 킹받아서 쫓아왔어요. 마누라 친구랑 하룻밤 재미봐놓고 들통나니 어쩔티비 내배째라? 언닌 그쪽 식구들한테 별의별 짓 다 당하면서 견디고 있는데, 이렇게 애매한 스탠스가 얼마나 사람 마음 힘들게 하는지 아세요? 욕 먹는 게 겁나요? 욕 한 바가지 드신다고 안 죽습니다! 사랑은 적어도 미안하단 말 하지 않는 게 기본 상식이에요. 이딴 것도 사랑이라고. 할많하않입니다 진짜."

In [114]:
sentence = "알잘딱깔센하게 처리합시다."

In [43]:
sentence = """
어제 친구랑 만나서 노잼 영화를 봤는데, 차라리 집에서 넷플릭스나 볼 걸 그랬어.
오늘 회사에서 팀장님께 혼나서 멘붕이 왔어.
이번 시험은 거의 망했어. 내가 공부할 땐 멍청하게도 뇌피셜로만 외웠거든.
주말에 친구들이랑 만나서 불금을 즐겼어.
저 사람은 언제 봐도 핵인싸야. 어딜 가나 인기 많아.
그 사람은 너무 대박이야! 모든 일을 척척 해내는 걸 보면 존경스러워.
이번 주말에는 집콕하면서 푹 쉬기로 했어.
친구가 요즘 완전 핵노잼이라 만나도 재미가 없어.
매일매일이 너무 힘들어서 힐링이 필요해.
그 가게 음식은 솔직히 비추야. 맛도 없고 서비스도 별로였어.
"""

### 1. 문장에서 신조어 찾은 후 인터넷에서 신조어의 뜻 검색

In [115]:
template = """
<<system>>
신조어란 *새로 만들어진* 단어 및 용어 가운데 표준어로 등재되지 않은 말을 뜻합니다.
당신은 신조어를 매우 잘 알고 있는 전문가로, [SENTENCE]에서 신조어 단어를 찾아 나열하고 있습니다.
신조어를 나열하는 것 이외에는 절대 대답하지 않습니다.
<</system>>
---
[SENTENCE]
{sentence}
[/SENTENCE]
"""

prompt_template = PromptTemplate.from_template(template)
chain = prompt_template | llm | StrOutputParser()

In [116]:
new_list = chain.invoke({"sentence": sentence}).split(", ")
context = []
for word in new_list:
    # context_eng = tavily.search(query="What is meaing " + word, search_depth="advanced")['results']
    context_kor = tavily.search(query=word + " 무슨 뜻", search_depth="advanced")['results']
    context += context_kor

print(context)

[{'title': '[한국어 신조어] 알잘딱깔센 뜻과 쓰임', 'url': 'https://nomadteacher.tistory.com/entry/한국어-신조어-알잘딱깔센-뜻과-쓰임', 'content': "긴말 하지 않아도 스스로 알아서 잘 하는 사람 말입니다. 이런 사람을 가리켜 표현하는 신조어가 있습니다. ' 알잘딱깔센 '인데요. 알잘딱깔센의 의미와 쓰임을 알아봅시다. 1. '알잘딱깔센'이란? 뜻과 유래. ' 알 아서 잘 딱 깔 끔하고 센 스있게'의 줄임말입니다 ...", 'score': 0.96429, 'raw_content': None}, {'title': '알잘딱깔센 뜻, 유래, 예시, 반대말까지 총정리', 'url': 'https://alltaste.tistory.com/entry/알잘딱깔센-뜻-유래-예시-반대말까지-총정리', 'content': '오늘의 줄임말은 "알잘딱깔센\'이다. 알잘딱깔센의 뜻 알아서 잘 딱 깔끔하고 센스 있게 를 줄인 말이라고 한다. 한마디로 \'알아서 센스 있게 잘\'이라는 의미이다. 알잘딱깔센 예시 의외로 사용하기는 쉽다. \'센스 있게\'가 들어갈 만한 자리에 넣어주면 오케이 ...', 'score': 0.95602, 'raw_content': None}, {'title': "Mz세대 '알잘딱깔센' 무슨 뜻? | 아주경제", 'url': 'https://www.ajunews.com/view/20200714145113127', 'content': "신조어 '알잘딱깔센'은 '알아서, 잘, 딱, 깔끔하고, 센스있게'를 줄인 신조어다. 알아서 잘 깔끔하고 센스있게 하라는 뜻으로 아나운서 김수현의 ...", 'score': 0.9369, 'raw_content': None}, {'title': '알잘딱깔센 - 나무위키', 'url': 'https://namu.wiki/w/알잘딱깔센', 'content': '실제로 알잘딱깔센 그 자체는 아무런 의미가 없는 단어들의 조합이다. 단어들이 가리키는 지향점은 최종적으로 센스 , 눈치 

In [118]:
new_list

['알잘딱깔센']

In [24]:
# # Quick hello world
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import PromptTemplate
# from langchain_upstage import ChatUpstage
# llm = ChatUpstage()

# prompt_template = PromptTemplate.from_template(
#     """
#     내가 드라마 대사를 줄테니 단어 하나마다 신조어로 추정되는지 되지 않는지 찾아서 나열해봐. 
#     나열하는 방법은 단어1이 신조어라면 (단어1: 신조어 맞음) 단어2가 신조어가 아니라면 (단어2: 신조어 아님)로 쭉 나열해줘.
#     ---------------------------------
#     {content}.
#     """
# )

# prompt_template.format(content="{text}")
# chain = prompt_template | llm | StrOutputParser()
# find_new_words_result = chain.invoke({"content": {example_text}})

### 2. 인터넷에서 찾은 뜻을 종합하여 신조어 뜻 사전 만들기

In [119]:
prompt_template = PromptTemplate.from_template(
    """
    내가 신조어가 들어있는 리스트와 리스트에 담긴 단어들을 검색한 정보를 줄테니 단어들의 뜻을 정보를 찾아 작성해줘.
    뜻을 찾아서 {{단어1: 뜻, 단어2: 뜻}}과 같은 딕셔너리 형태로 대답해.   
    ---------------------------------
    
    {content}.
    {context}
    """
)

chain = prompt_template | llm | StrOutputParser()
new_words_meaning = chain.invoke({"content": {str(new_list)}, "context" : {str(context)}})
print(new_words_meaning)

알잘딱깔센: "알아서 잘 딱 깔끔하고 센스있게"의 줄임말로, 어떤 일을 스스로 알아서 잘 처리하는 것을 의미합니다.


### 3. 사전을 바탕으로 문장 수정

In [120]:
prompt_template = PromptTemplate.from_template(
    """
    내가 텍스트와 텍스트 속 바꿀 단어와 각 단어의 뜻을 줄테니, 단어 뜻을 참고해 문장을 크게 바꾸지 않고, 문장의 어투를 바꾸지 않고, 바꿀 단어만 자연스럽게 문장을 수정해서 보여줘.
    그래서 원본 텍스트와 수정된 텍스트를 각각 나열해줘
    ---------------------------------
    원본 텍스트 : {sentence}
    텍스트 속 바꿀 단어와 뜻: {new_words_meaning}
    """
)

chain = prompt_template | llm | StrOutputParser()
result2 = chain.invoke({"sentence": {sentence}, "new_words_meaning":{new_words_meaning}})
print(result2)

원본 텍스트: 알잘딱깔센하게 처리합시다.
수정된 텍스트: 스스로 알아서 잘 처리하고, 깔끔하고 센스있게 처리합시다.
